# Text als Struktur

## Vorbemerkung

Textdaten können in unterschiedlichen Formen und Formaten vorliegen. In der computerbasierten Analyse ist mit Text zunächst der »reine Text« gemeint, also Buchstabenfolgen ohne Formatierungen wie Fettdruck oder Textgliederungen wie Überschriften. Diese sind in Python einfache Zeichenketten bzw. Strings.

Aus historischen Gründen – Amerika war früh dominierend in der Computertechnologie, Amerikaner benutzen ein lateinisches Alphabet ohne Umlaute – ist die Unterstützung für Sonderzeichen und nicht-westliche Sprachen vergleichsweise spät vereinheitlicht worden. Die Unterstützung für nahezu alle Sprachen der Welt ist im sogenannten Unicode-Standard geregelt. In Python 2 müssen dafür Zeichenketten das Präfix *u* erhalten. In Python 3 ist die Unterstützung dafür standardmäßig aktiviert.

In [1]:
# Ohne Präfix können merkwürdige Effekte auftreten:
len('Tür')

4

In [2]:
# Mit Präfix funktioniert es:
len(u'Tür')

3

## Mehr als Text

Um Texten zusätzliche Informationen wie etwa Textformatierungen oder Textstruktur hinzuzufügen, muss man den Text um Zusatzinformationen anreichern. Das ist die Aufgabe von Textformaten wie z.B. *doc*, *odt* oder *html*. [HTML](http://de.wikipedia.org/wiki/Hypertext_Markup_Language) ist dabei die »Sprache des Internet« und vergleichsweise einfach aufgebaut.

In [3]:
from IPython.core.display import HTML
HTML(u'Meinst du <b>diese</b> Tür?')

Viele Daten liegen im Internet als HTML-Dateien vor. Diese sind für die Sozialwissenschaften häufig relevantes Analysematerial. Dabei müssen in vielen Fällen zunächst die relevanten Daten aus Internetseiten extrahiert werden. So beschreibt die Wikipedia das sogenannte [Screen Scraping](http://de.wikipedia.org/wiki/Screen_Scraping#Funktionsweise):

> Screen Scraping besteht im Wesentlichen aus zwei Schritten:
>
> * Abrufen von Webseiten
> * Extraktion der relevanten Daten


HTML-Dokumente können als Baumstruktur beschrieben werden:

In [4]:
from lxml import html
doc = html.fromstring(u'''
<html>
  <head>
    <title>Alles &#252;ber T&#252;ren</title>
  </head>
  <body>
    <div id="content">
      <h1>Welche T&#252;r passt zu mir?</h1>
      <p class="teaser">Das k&#246;nnen Sie auf <b>dieser Seite</b> herausfinden.</p>
      <p>Machen Sie unseren <a href="test.html">Test!</a></p>
    </div>
    <div id="nav">
      <ul>
        <li><a href="fenster.html">Fenster</a></li>
        <li><a href="luken.html">Luken</a></li>
      </ul
    </div>
  </body>
</html>
''')

In [5]:
for element in doc.iter():
    indent = len(list(element.iterancestors()))
    print '{space}- {tag}'.format(space=u'  ' * indent,
                                  tag=element.tag)

- html
  - head
    - title
  - body
    - div
      - h1
      - p
        - b
      - p
        - a
    - div
      - ul
        - li
          - a
        - li
          - a


Aus dieser Baumstruktur lassen sich gezielt bestimmte Informationen extrahieren. Dazu gibt es zwei Methoden: XPath und CSS.

### XPath

Die Ordnerstruktur einer Festplatte oder die Navigation einer Website lässt sich auch als Baum beschreiben, wie etwa im [Inhaltsverzeichnis](https://www.unilu.ch/inhaltsverzeichnis/) der Uni Luzern. Ein bestimmtes Element in diesem Baum lässt sich über einen Pfad beschreiben:

![Website des Soziologischen Seminars](Bilder/UniLU.png)

Ähnlich funktioniert dies auch mit dem HTML-Dokument. Dabei können aber an einer Adresse mehrere Elemente sitzen, eine Adresse gibt daher immer eine Liste von passenden Elementen zurück:

In [6]:
doc.xpath('/html/body/div/p')

[<Element p at 0x7f2538202db8>, <Element p at 0x7f2538202f18>]

In [7]:
doc.xpath('/html/body/div/h1/text()')

[u'Welche T\xfcr passt zu mir?']

In [8]:
# Aber:
doc.xpath('/html/body/div/p/text()')

[u'Das k\xf6nnen Sie auf ', ' herausfinden.', 'Machen Sie unseren ']

In [9]:
# Daher besser:
[p.text_content() for p in doc.xpath('/html/body/div/p')]

[u'Das k\xf6nnen Sie auf dieser Seite herausfinden.',
 'Machen Sie unseren Test!']

In [10]:
# Kurzformen
doc.xpath('//p')

[<Element p at 0x7f2538202db8>, <Element p at 0x7f2538202f18>]

Reale Webseiten enthalten oft eine Menge zusätzlicher Elemente, die für die Textanalyse störend sind, wie z.B. Navigation, Anzeigen, Kopfzeilen und dekorative Elemente. Um gezielt die relevanten Passagen zu finden, muss man die Pfade oft mit zusätzlichen Filterbedingungen einschränken. Typische Kriterien können hier die HTML-Attribute *id* und *class* sein, die oft zur Strukturierung von Webseiten eingesetzt werden. Hier ist XPath sehr vielseitig:

In [11]:
doc.xpath('//p[@class="teaser"]')

[<Element p at 0x7f2538202db8>]

In [12]:
doc.xpath('//div[@id="nav"]//a')

[<Element a at 0x7f2538192418>, <Element a at 0x7f2538202e68>]

### CSS

Die Syntax für diese Bedingungen ist zwar sehr flexibel, aber auch recht kompliziert. In den meisten Fällen bietet sich eine einfachere Variante an, die auf die Besonderheiten von HTML zugeschnitten ist: CSS-Selektoren. [CSS](http://de.wikipedia.org/wiki/Cascading_Style_Sheets) ist eigentlich eine Sprache für die Formatierung von Webseiten. Sie enthält aber eine Möglichkeit, Elemente in HTML-Dokumenten auszuwählen, die sich auch in anderen Kontexten einsetzen lässt. Ihre Hauptelemente sind dabei Tag-Namen, IDs und Klassen.

In [13]:
doc.cssselect('a')

[<Element a at 0x7f79e00848e8>,
 <Element a at 0x7f79e0065310>,
 <Element a at 0x7f79e00d5e68>]

In [14]:
# IDs (z.B. <div id="nav">)
doc.cssselect('#nav a')

[<Element a at 0x7f79e0065310>, <Element a at 0x7f79e00d5e68>]

In [15]:
# Klassen (z.B. <p class="teaser">)
doc.cssselect('p.teaser')  # oder nur '.teaser'

[<Element p at 0x7f79e00651b0>]

Eine ausführliche Dokumentation der unterschiedlichen Selektoren findet sich [hier](http://wiki.selfhtml.org/wiki/CSS/Selektoren). Ein Hilfsmittel, um CSS-Selektoren zu finden, ist [Selector Gadget](http://selectorgadget.com/).

## Ein echtes Beispiel

Nehmen wir als Beispiel eine [Rede der deutschen Bundeskanzlerin](http://www.bundesregierung.de/Content/DE/Rede/2014/09/2014-09-17-merkel-jugend-forscht.html) von der Website der deutschen Bundesregierung. Wir wollen den Titel, das Datum, den Teaser und den Text dieser Seite für weitere Analysen extrahieren.

In [16]:
rede = html.parse('http://www.bundesregierung.de/Content/DE/Rede/2014/09/2014-09-17-merkel-jugend-forscht.html')
root = rede.getroot()

In [17]:
# Titel
root.cssselect('.text h1')[0].text_content()

u'Rede von Bundeskanzlerin Merkel zum Empfang der Preistr\xe4gerinnen und Preistr\xe4ger des 49. Bundeswettbewerbs \u201eJugend forscht\u201c am 17. September 2014'

In [18]:
# Teaser
root.cssselect('.text .abstract')[0].text_content()

'im Bundeskanzleramt\n'

Das Datum ist etwas komplizierter: Diese Kopfangaben auf der Seite bestehen aus Definitionslisten ([dl](http://wiki.selfhtml.org/wiki/HTML/Textstrukturierung/dl)) mit dem folgenden Aufbau:

```html
<dl>
    <dt>Datum:</dt><dd>17. September 2014</dd>
    <dt>Ort:</dt><dd>Berlin</dd>
</dl>
```

In diesem Fall ist nur das Datum in dieser Form angegeben, aber bei anderen Reden ist dies unterschiedlich. In Python lässt sich das Datum so herausfiltern:

In [19]:
# Datum
labels = root.cssselect('.text dl dt')
contents = root.cssselect('.text dl dd')
date = None
for label, content in zip(labels, contents):
    if 'Datum' in label.text_content():
        date = content.text_content()
        break
date

'17. September 2014'

In [20]:
# Text
pars = root.cssselect('.text .abstract ~ p')  # Alle p-Elemente hinter .abstract
text = '\n\n'.join([par.text_content() for par in pars])
text[0:200]

u'Lieber Herr Baszio,\nliebe Frau Wanka,\nmeine Damen und Herren,\naber vor allem: liebe Preistr\xe4gerinnen und Preistr\xe4ger von \u201eJugend forscht\u201c,\n\nich habe Ihnen einen der spannenden Termine im Kalenderjahr '

Auf diese Weise lassen sich nun die relevanten Inhalte aller Seiten herausfiltern. In der Praxis würde man auch hier nicht Seite für Seite einzeln aufrufen, sondern auf ähnliche Weise auf einer Übersichtsseite alle Links zu den einzelnen Texten herausfiltern und diese automatisch abarbeiten. Für größere Projekte bietet sich [Scrapy](http://doc.scrapy.org/en/latest/intro/tutorial.html) an, das die Ergebnisse direkt in einer Tabelle speichern kann. Im Ordner *politikdokumente* findet sich ein Beispiel für ein solches Scrapy-Projekt.